In [12]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt


def filtering_2d(image, kernel):
    image_height, image_width = image.shape
    kernel_height, kernel_width = kernel.shape
    
    # Calculate the output dimensions
    output_height = image_height - kernel_height + 1
    output_width = image_width - kernel_width + 1
    
    # Initialize the output matrix
    output = np.zeros((output_height, output_width))
    
    # Apply convolution (slide the kernel over the image)
    for i in range(output_height):
        for j in range(output_width):
            # Element-wise multiplication and sum
            output[i, j] = np.sum(image[i:i+kernel_height, j:j+kernel_width] * kernel)
    
    return output

def get_x_derivative(img_matrix, kernel):
    pass


def get_y_derivative(img_matrix, kernel):
    pass


def get_grad_mag():
    pass


def get_grad_angle(matrix_a, matrix_b):
    pass

In [13]:
group_img = Image.open("dataset/group-photo.jpeg")
gray_group_img = group_img.convert('L')

# Convert the image to a NumPy array
img_matrix = np.array(gray_group_img)

# Display the shape and type of the array
print(img_matrix.shape)
print(img_matrix.dtype)
print("img_matrix", img_matrix) 

# step 1: Apply Gaussian smoothing on the image.
# 3x3 Gaussian Fiter kernel
gaussian_kernel = np.array([[1, 2, 1], [2, 4, 2], [1, 2, 1]])
gaussian_filtering = filtering_2d(img_matrix, gaussian_kernel)

print("gaussian_filtering", gaussian_filtering)

# step 2a: Get X derivative

# step 2b: Get Y derivative

(1066, 1600)
uint8
img_matrix [[124 124 124 ... 110 110 110]
 [124 124 124 ... 110 110 110]
 [124 124 124 ... 110 110 110]
 ...
 [153 156 160 ... 196 196 196]
 [153 156 160 ... 196 196 196]
 [153 156 160 ... 196 196 196]]
gaussian_filtering [[1984. 1984. 1984. ... 1760. 1760. 1760.]
 [1984. 1984. 1984. ... 1760. 1760. 1760.]
 [1980. 1980. 1980. ... 1760. 1760. 1760.]
 ...
 [2500. 2557. 2608. ... 3136. 3136. 3136.]
 [2500. 2555. 2593. ... 3136. 3136. 3136.]
 [2500. 2553. 2581. ... 3136. 3136. 3136.]]
